<a href="https://colab.research.google.com/github/Naidu-4444/langgraph-chatbot-with-memory/blob/main/Langgraph_with_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 16.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [ ]:
!pip install langchain_core

In [ ]:
TAVILY_KEY=userdata.get('TAVILY_API_KEY')

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(tavily_api_key=TAVILY_KEY)
tools_list = [search]

/tmp/ipython-input-4069025217.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(tavily_api_key=TAVILY_KEY)


In [ ]:
from langchain.agents import initialize_agent
from langchain_community.tools import TavilySearchResults
search = TavilySearchResults(tavily_api_key=TAVILY_KEY)
tools_list = [search]

agent_with_search_tool = initialize_agent(tools = tools_list, llm = model, verbose=True, agent = "zero-shot-react-description")

agent_with_search_tool.invoke("Who won the Champions Trophy in 2025?")



> Entering new AgentExecutor chain...
I need to find out which team won the Champions Trophy in 2025. I will use the search tool to find the answer.
Action: tavily_search_results_json
Action Input: "Champions Trophy 2025 winner"
Observation: [{'title': '2025 ICC Champions Trophy final - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/2025_ICC_Champions_Trophy_final', 'content': '| New Zealand | India |\n| --- | --- |\n| New Zealand | India |\n| 251/7 | 254/6 |\n| 50 | 49 |\n\nNew Zealand\nIndia\n\nThe 2025 ICC Champions Trophy final was a One Day International (ODI) cricket match played at the Dubai International Cricket Stadium on 9 March 2025 to determine the winner of 2025 ICC Champions Trophy. It was played between India and New Zealand. It was the second time that India and New Zealand played a Champions Trophy final against each other, after the 2000 final. [...] |  |  |\n| --- | --- |\n| Match programme cover | |\n| Event | 2025 ICC Champions Trophy |\n| | New Zealand | Indi

{'input': 'Who won the Champions Trophy in 2025?',
 'output': 'India won the Champions Trophy in 2025.'}

In [ ]:
from langgraph.graph import StateGraph,END,add_messages
from langchain_core.messages import HumanMessage, AIMessage
from typing import TypedDict,Annotated
from langgraph.checkpoint.memory import MemorySaver

class MyState(TypedDict):
  messages: Annotated[list,add_messages]

graph = StateGraph(MyState)

memory=MemorySaver()

def chatbot(state : MyState):
  res=model.invoke(state["messages"])
  return {"messages" : [res]}

graph.add_node("chat",chatbot)
graph.set_entry_point("chat")
graph.add_edge("chat",END)

agent=graph.compile(checkpointer=memory)

configuration1={
    "configurable":{
        "thread_id":1
    }
}

Query_from_Ram_1=agent.invoke({"messages" : [HumanMessage(content="My name is Ram")]},config=configuration1)

print(Query_from_Ram_1)

{'messages': [HumanMessage(content='My name is Ram', additional_kwargs={}, response_metadata={}, id='ed002bcd-2a2f-495f-b9ea-b158457e4bed'), AIMessage(content="Okay, Ram! It's nice to meet you. Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b6494470-2100-4402-b399-ddcc14665d20-0', usage_metadata={'input_tokens': 4, 'output_tokens': 23, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})]}


In [ ]:
Query_from_Ram_2=agent.invoke({"messages" : [HumanMessage(content="What is my name?")]},config=configuration1)
print(Query_from_Ram_2)

{'messages': [HumanMessage(content='My name is Ram', additional_kwargs={}, response_metadata={}, id='ed002bcd-2a2f-495f-b9ea-b158457e4bed'), AIMessage(content="Okay, Ram! It's nice to meet you. Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b6494470-2100-4402-b399-ddcc14665d20-0', usage_metadata={'input_tokens': 4, 'output_tokens': 23, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='09c37ff7-6668-49d1-8cee-36f81d3b00e6'), AIMessage(content='Your name is Ram.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--157cc889-ef5c-41a2-80b0-3b8868b459fc-0', u

In [ ]:
configuration2={
    "configurable":{
        "thread_id":2
    }
}

Query_from_Gopal_1=agent.invoke({"messages" : [HumanMessage(content="My name is Gopal")]},config=configuration2)

print(Query_from_Gopal_1)

{'messages': [HumanMessage(content='My name is Gopal', additional_kwargs={}, response_metadata={}, id='929b97b9-f5ce-4b50-80fe-2a642f000eef'), AIMessage(content="Okay, Gopal! It's nice to meet you (or connect with you again, if we've interacted before). How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--96e353b3-a84c-4b1d-9e73-d323b8dbc838-0', usage_metadata={'input_tokens': 4, 'output_tokens': 33, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})]}


In [ ]:
Query_from_Gopal_2=agent.invoke({"messages" : [HumanMessage(content="what is my name?")]},config=configuration2)

print(Query_from_Gopal_2)

{'messages': [HumanMessage(content='My name is Gopal', additional_kwargs={}, response_metadata={}, id='929b97b9-f5ce-4b50-80fe-2a642f000eef'), AIMessage(content="Okay, Gopal! It's nice to meet you (or connect with you again, if we've interacted before). How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--96e353b3-a84c-4b1d-9e73-d323b8dbc838-0', usage_metadata={'input_tokens': 4, 'output_tokens': 33, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='e8c097c3-feb6-4102-8d92-0ef0edf1a355'), AIMessage(content='Your name is Gopal.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-